In [1]:
from skimage.io import imread_collection
from skimage.transform import resize,rotate
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from skimage import exposure
from scipy import ndimage
from skimage import util

def load_images(class_name, values, labels):
    path_to_image = 'raw-img/'
    postfix = '/*.jp*'
    image_size = (64,64,3)
    i = 0
    for image in imread_collection(path_to_image + class_name + postfix):
        
        img = resize(image, image_size)
        values.append(img)
        labels.append(class_name)
        values.append(exposure.adjust_sigmoid(img))
        labels.append(class_name)
        values.append(exposure.adjust_gamma(img))
        labels.append(class_name)
        values.append(ndimage.uniform_filter(img,(2,2,1)))
        labels.append(class_name)
        values.append(rotate(img, -30))
        labels.append(class_name)
        values.append(rotate(img, 30))
        labels.append(class_name)
        values.append(rotate(img, -60))
        labels.append(class_name)
        values.append(util.random_noise(img))
        labels.append(class_name)

def create_data_set():
    class_names = ['spider', 'horse', 'elephant', 'chicken', 'sheep']
    values = []
    labels = []
    for class_name in class_names:
        load_images(class_name, values, labels)

    return np.array(values), np.array(labels)

x, y = create_data_set()
 
class HogTransform(BaseEstimator, TransformerMixin):
    def __init__(self, y=None, orientations=9,
                 pixels_per_cell=(8, 8),
                 cells_per_block=(3, 3), block_norm='L2-Hys'):
        self.y = y
        self.orientations = orientations
        self.pixels_per_cell = pixels_per_cell
        self.cells_per_block = cells_per_block
        self.block_norm = block_norm
 
    def fit(self, X, y=None):
        return self
 
    def transform(self, X, y=None):
        return np.array([ 
            hog(img,
                orientations=self.orientations,
                pixels_per_cell=self.pixels_per_cell,
                cells_per_block=self.cells_per_block, channel_axis=-1) for img in X ])


/home/wiktor/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper
/home/wiktor/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:293: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  plugin_module = __import__('skimage.io._plugins.' + modname,


In [2]:
hog_pipeline = Pipeline([
    ('hog_transform', HogTransform()),
    ('scaler', StandardScaler()),
    ('logistic',KNeighborsClassifier(n_neighbors=6))
])
 
param_grid = [
    {
        'hog_transform__orientations': [4, 5, 6],
        'hog_transform__cells_per_block': [(8,8) (7, 7), (6,6)],
        'hog_transform__pixels_per_cell': [(8,8), (7,7), (6, 6)]
    },

]
search = GridSearchCV(hog_pipeline, param_grid, n_jobs=2, verbose=10)
search.fit(x, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
[CV 1/5; 1/27] START hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6)
[CV 2/5; 1/27] START hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6)
[CV 1/5; 1/27] END hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6);, score=0.566 total time=   3.7s
[CV 3/5; 1/27] START hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6)
[CV 2/5; 1/27] END hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6);, score=0.579 total time=   4.2s
[CV 4/5; 1/27] START hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform__pixels_per_cell=(6, 6)
[CV 3/5; 1/27] END hog_transform__cells_per_block=(2, 2), hog_transform__orientations=7, hog_transform

In [9]:
from sklearn import svm

scalify = StandardScaler()
hog_trasform = HogTransform(orientations = 5, pixels_per_cell=(8,8), cells_per_block=(8,8))

x_train = scalify.fit_transform(hog_trasform.transform(x))

svm_pipeline = Pipeline([
    ('logistic',svm.SVC())
])
 
param_grid = [
    {
        'logistic__C': [ 0.4, 0.5, 0.8, 1.1, 2, 3, 5, 6]
    },

]
search = GridSearchCV(svm_pipeline, param_grid, n_jobs=2, verbose=10)
search.fit(x_train, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV 1/5; 1/8] START logistic__C=0.4.............................................
[CV 2/5; 1/8] START logistic__C=0.4.............................................
[CV 1/5; 1/8] END ..............logistic__C=0.4;, score=0.697 total time= 2.4min
[CV 3/5; 1/8] START logistic__C=0.4.............................................
[CV 2/5; 1/8] END ..............logistic__C=0.4;, score=0.706 total time= 2.4min
[CV 4/5; 1/8] START logistic__C=0.4.............................................
[CV 3/5; 1/8] END ..............logistic__C=0.4;, score=0.736 total time= 1.8min
[CV 5/5; 1/8] START logistic__C=0.4.............................................
[CV 4/5; 1/8] END ..............logistic__C=0.4;, score=0.714 total time= 1.9min
[CV 1/5; 2/8] START logistic__C=0.5.............................................
[CV 5/5; 1/8] END ..............logistic__C=0.4;, score=0.691 total time= 1.7min
[CV 2/5; 2/8] START logistic__C=0.5..............

In [7]:
from sklearn.neural_network import MLPClassifier


scalify = StandardScaler()
hog_trasform = HogTransform(orientations = 5, pixels_per_cell=(8,8), cells_per_block=(8,8))

x_train = scalify.fit_transform(hog_trasform.transform(x))

mlp_pipeline = Pipeline([
    ('logistic', MLPClassifier())
])
 
param_grid = [
    {
        'logistic__alpha': np.logspace(-3, 2, 15)
    }

]
search = GridSearchCV(mlp_pipeline, param_grid, n_jobs=2, verbose=10)
search.fit(x_train, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


Fitting 5 folds for each of 15 candidates, totalling 75 fits
[CV 1/5; 1/15] START logistic__alpha=0.001......................................
[CV 2/5; 1/15] START logistic__alpha=0.001......................................


In [5]:
from sklearn.neighbors import KNeighborsClassifier

scalify = StandardScaler()
hog_trasform = HogTransform(orientations = 5, pixels_per_cell=(8,8), cells_per_block=(8,8))

x_train = scalify.fit_transform(hog_trasform.transform(x))

knn_pipeline = Pipeline([
    ('logistic', KNeighborsClassifier())
])
 
param_grid = [
    {
        'logistic__n_neighbors': np.arange(50)
    },

]
search = GridSearchCV(knn_pipeline, param_grid, n_jobs=2, verbose=10)
search.fit(x_train, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)


Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5; 1/100] START logistic__n_neighbors=0...................................
[CV 2/5; 1/100] START logistic__n_neighbors=0...................................
[CV 1/5; 1/100] END ......logistic__n_neighbors=0;, score=nan total time=   2.0s
[CV 2/5; 1/100] END ......logistic__n_neighbors=0;, score=nan total time=   2.0s
[CV 3/5; 1/100] START logistic__n_neighbors=0...................................
[CV 4/5; 1/100] START logistic__n_neighbors=0...................................
[CV 4/5; 1/100] END ......logistic__n_neighbors=0;, score=nan total time=   0.6s
[CV 3/5; 1/100] END ......logistic__n_neighbors=0;, score=nan total time=   0.6s
[CV 5/5; 1/100] START logistic__n_neighbors=0...................................
[CV 1/5; 2/100] START logistic__n_neighbors=1...................................
[CV 5/5; 1/100] END ......logistic__n_neighbors=0;, score=nan total time=   0.4s
[CV 2/5; 2/100] START logistic__n_neighbors=1.

/home/wiktor/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
5 fits failed out of a total of 500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/home/wiktor/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/wiktor/.local/lib/python3.8/site-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/home/wiktor/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py", line 198, in fit
    return self._fit(X, y)
  File